### Dependencies

In [1]:
import math
import numpy as np
import random
import pandas 
import plotly.express as px

### Functions

In [16]:
# return Gumbel's function value
def gumbel(x):
    return math.pow(math.e, x) * math.pow(math.e, -math.pow(math.e, x))

# return an array tuple of randomly sampled datapoints
def random_points(start, end, n_points):
    x_datapoints = sorted([start+(end-start)*random.random() for _ in range(n_points)])
    y_datapoints = list(map(gumbel, x_datapoints))
    return np.array(x_datapoints), np.array(y_datapoints)

# return an array tuple of equally distanced datapoints
def equal_points(start, end, n_points):
    x_datapoints = [_ for _ in range(start, end+1, 1)]
    y_datapoints = list(map(gumbel, x_datapoints))
    return np.array(x_datapoints), np.array(y_datapoints)

# return an array tuple of datapoints given by the Chebyshev points
def cheby_points(start, end, n_points):
    x_datapoints = [start+(end-start)*((-math.cos((x-1)*math.pi/n_points)+1)/2) for x in range(n_points)]
    y_datapoints = list(map(gumbel, x_datapoints))  
    return np.array(x_datapoints), np.array(y_datapoints)

# return an array of coefficients using Newton-Rhapson's divided differences
def nr_coeffts(x_datapoints, y_datapoints, n_points):
    a = y_datapoints.copy()
    for k in range(1, n_points):
        a[k:n_points] = (a[k:n_points] - a[k-1])/(x_datapoints[k:n_points] - x_datapoints[k-1])
    return a

# return the function value given by the interpolated function via the Newton-Rhapson method
def nr_evalfunct(coefficients, x_datapoints, x, degree):
    value = coefficients[degree]
    for k in range(1, degree+1):
        value = coefficients[degree-k] + (x - x_datapoints[degree-k])*value
    return value

# return the reconstruction error of the polynomial interpolation
def poly_recon_error(x_datapoints, y_datapoints, step_size, start, end, n_points):
    coeffts = nr_coeffts(x_datapoints, y_datapoints, n_points)
    error = 0
    for x in range(start*10, (end)*10+1, step_size):
       error += abs(gumbel(x/10) - nr_evalfunct(coeffts, x_datapoints, x/10, n_points-1))
    return error

# plot the polynomial interpolation
def plot_interp(x_datapoints, y_datapoints, step_size, start, end, n_points):
    coeffts = nr_coeffts(x_datapoints, y_datapoints, n_points)
    xdata = []
    ydata = []
    for x in range(start*10, (end)*10+1, step_size):
       xdata.append(x/10)
       ydata.append(nr_evalfunct(coeffts, x_datapoints, x/10, n_points-1))
    fig = px.scatter(x=xdata, y=ydata)
    fig.show()

# plot the gumbel function
def plot_gumbel(step_size, start, end):
    xdata = []
    ydata = []
    for x in range(start*10, (end)*10+1, step_size):
       xdata.append(x/10)
       ydata.append(gumbel(x/10))
    fig = px.scatter(x=xdata, y=ydata)
    fig.show()

def radial_recon_error(x_datapoints, y_datapoints, step_size, start, end):
    ...

In [17]:
start = -5
end = 3
n_points = 9
step_size = 1   

random_x_datapoints, random_y_datapoints = random_points(start, end, n_points)
fig = px.scatter(x=random_x_datapoints, y=random_y_datapoints)
fig.show()

equal_x_datapoints, equal_y_datapoints = equal_points(start, end, n_points)
fig = px.scatter(x=equal_x_datapoints, y=equal_y_datapoints)
fig.show()

cheby_x_datapoints, cheby_y_datapoints = cheby_points(start, end, n_points)
fig = px.scatter(x=cheby_x_datapoints, y=cheby_y_datapoints)
fig.show()

In [18]:
print(cheby_x_datapoints)
print(cheby_y_datapoints)

[-4.75877048 -5.         -4.75877048 -4.06417777 -3.         -1.69459271
 -0.30540729  1.          2.06417777]
[0.00850291 0.0066927  0.00850291 0.01688457 0.04736901 0.15285481
 0.35266718 0.17937408 0.00298356]


In [19]:
print(poly_recon_error(random_x_datapoints, random_y_datapoints, step_size, start, end, n_points))
print(poly_recon_error(equal_x_datapoints, equal_y_datapoints, step_size, start, end, n_points))
print(poly_recon_error(cheby_x_datapoints, cheby_y_datapoints, step_size, start, end, n_points))
# ^ problem nagkakaroon ng 0/0 sa divided differences

4.704756773223791
1.4152350082555476
nan


/var/folders/7v/lh4dpmwx58b8f4ly3bvf48000000gn/T/ipykernel_7499/2685343390.py:27: RuntimeWarning:

invalid value encountered in divide



In [9]:
# plot_interp(random_x_datapoints, random_y_datapoints, step_size, start, end, n_points)
# plot_interp(equal_x_datapoints, equal_y_datapoints, step_size, start, end, n_points)
# plot_gumbel(step_size, start, end)